<a href="https://colab.research.google.com/github/AdriMello/Avaliacao-Regressao-Linear-e-Polinomial/blob/main/C%C3%B3pia_de_Transcricaoipynb_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-whisper
!pip install whisper
!pip install SpeechRecognition
!pip install --upgrade whisper
!pip install --upgrade librosa

In [ ]:
import whisper
import re
from google.colab import files
import os
import markdown
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Criar diretório para salvar transcrições
os.makedirs('transcricoes', exist_ok=True)

# Módulo de transcrição e análise de áudio
def transcrever_e_analisar_audio(arquivo_de_audio):
    try:
        modelo = whisper.load_model("small")
        resultado = modelo.transcribe(arquivo_de_audio)
        texto = resultado["text"]
        duracao = resultado["segments"][-1]["end"]

        # Melhorar a formatação
        texto = markdown.markdown(texto)

        # Remover ruídos
        texto = re.sub(r"Tá|Ah", "", texto)

        return texto, duracao
    except Exception as e:
        print(f"Erro ao transcrever áudio: {e}")
        return None, None

# Função para salvar a transcrição em um arquivo
def salvar_transcricao(texto, nome_arquivo):
    with open(f'transcricoes/{nome_arquivo}', 'w', encoding='utf-8') as f:
        f.write(texto)

arquivos_de_audio_uploadados = []
# Selecionar arquivo de áudio
def selecionar_arquivo():
    global arquivos_de_audio_uploadados
    if not arquivos_de_audio_uploadados:
        uploaded = files.upload()
        arquivos_de_audio_uploadados = list(uploaded.keys())
        print("Arquivos de áudio uploadados:")
        for i, arquivo in enumerate(arquivos_de_audio_uploadados):
            print(f"{i+1}. {arquivo}")
    return arquivos_de_audio_uploadados

# Main
arquivos_de_audio_uploadados = selecionar_arquivo()
if arquivos_de_audio_uploadados:
    for arquivo_de_audio in arquivos_de_audio_uploadados:
        texto, duracao = transcrever_e_analisar_audio(arquivo_de_audio)
        if texto:
            texto_formatado = re.sub(r'([.!?])', r'\1\n', texto)
            # print("Texto Transcrito:")
            # print(texto_formatado)

            # # Salvar a transcrição em um arquivo
            # nome_arquivo = arquivo_de_audio.replace('.mp3', '_transcricao.txt').replace('.wav', '_transcricao.txt')
            # salvar_transcricao(texto_formatado, nome_arquivo)
            # print(f"Transcrição salva em: {nome_arquivo}")
        else:
            print("Erro ao transcrever o áudio.")
else:
  print("Erro ao carregar o arquivo de áudio.")

Saving 2874774.wav to 2874774 (1).wav
Saving 2874830.wav to 2874830 (1).wav
Saving 2961972.wav to 2961972.wav
Saving 2962046.wav to 2962046.wav
Saving 2962074.wav to 2962074.wav
Arquivos de áudio uploadados:
1. 2874774 (1).wav
2. 2874830 (1).wav
3. 2961972.wav
4. 2962046.wav
5. 2962074.wav
Transcrição salva em: 2874774 (1)_transcricao.txt
Transcrição salva em: 2874830 (1)_transcricao.txt
Transcrição salva em: 2961972_transcricao.txt
Transcrição salva em: 2962046_transcricao.txt
Transcrição salva em: 2962074_transcricao.txt


In [ ]:
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Função para processar texto
def processar_texto(texto):
    lemmatizer = WordNetLemmatizer()
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)  # Remove números
    palavras = texto.split()
    palavras = [palavra for palavra in palavras if palavra not in stopwords.words('portuguese')]
    palavras = [lemmatizer.lemmatize(palavra) for palavra in palavras]
    return ' '.join(palavras)

# Função para carregar transcrições de arquivos
def carregar_transcricoes(diretorio):
    transcricoes = []
    arquivos = os.listdir(diretorio)
    for arquivo in arquivos:
        if arquivo.endswith('_transcricao.txt'):
            with open(os.path.join(diretorio, arquivo), 'r', encoding='utf-8') as f:
                transcricoes.append(f.read())
    return transcricoes

# Função para rotular automaticamente com base nas palavras-chave
def rotular_transcricao(texto, limiar_promotor=1, limiar_detrator=1):
    contagem_promotor = contar_palavras(texto, palavras_promotor)
    contagem_detrator = contar_palavras(texto, palavras_detrator)
    total_promotor = sum(contagem_promotor.values())
    total_detrator = sum(contagem_detrator.values())
    if total_promotor >= limiar_promotor and total_promotor > total_detrator:
        return "promotor"
    elif total_detrator >= limiar_detrator and total_detrator > total_promotor:
        return "detrator"
    else:
        return "neutro"

# Função para contar palavras de destaque
def contar_palavras(texto, palavras):
    contagem = {palavra: texto.lower().count(palavra) for palavra in palavras}
    return contagem

# Função para realizar a rotulagem
def realizar_rotulagem(diretorio):
    transcricoes = carregar_transcricoes(diretorio)
    rotulos = []

    for texto in transcricoes:
        texto_processado = processar_texto(texto)
        rotulo = rotular_transcricao(texto_processado)
        rotulos.append(rotulo)

    # Criar o DataFrame
    df = pd.DataFrame({'Transcrição': transcricoes, 'Rótulo': rotulos})
    print("\nResultados:\n", df)

# Definir palavras de destaque para promotores e detratores
palavras_promotor = [
    "ótimo", "excelente", "muito bom", "bom", "satisfeito", "recomendo",
    "gostei", "amor", "paixão", "orgulho", "satisfação", "facilidade", "eficiência",
    "boa experiência", "produto incrível", "suporte excelente"
]
palavras_detrator = [
    "ruim", "péssimo", "muito ruim", "desapontado", "insatisfeito", "não recomendo",
    "não gostei", "ódio", "desgosto", "desapontamento", "descontentamento",
    "dificuldade", "inconveniente", "problema", "erro", "falta de suporte"
]

# Realizar a rotulagem
realizar_rotulagem('transcricoes')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Resultados:
                                          Transcrição    Rótulo
0  <p>Alô, Alô.\n Fala com o Sr.\n Pedro Rogério....  detrator
1  <p>Bom dia, eu falo com o Sr.\n Pedro.\n Bom d...  promotor
2  <p>Alô, bom dia.\n Alô, bom dia.\n Alô?\n Alô,...  promotor
3  <p>Bom dia, eu falo com o Sr.\n Pedro.\n Bom d...  promotor
4  <p>Depende de Cláudia.\n Oi Cláudia, bom dia.\...  detrator
5  <p>Candinavia, Natalia, bom dia.\n Bom dia, Na...  promotor
6  <p>Candinavia, Natalia, bom dia.\n Bom dia, Na...  promotor


In [ ]:
from collections import Counter

# Módulo de estatísticas
def calcular_estatisticas(texto):
    texto_limpo = re.sub(r'[^\w\s]', '', texto).lower()
    total_palavras = len(texto_limpo.split())
    palavras_mais_comuns = Counter(texto_limpo.split()).most_common(10)
    comprimento_medio_palavras = sum(len(palavra) for palavra in texto_limpo.split()) / total_palavras
    contagem_pontuacao = len(re.findall(r'[^\w\s]', texto))

    return {
        'total_palavras': total_palavras,
        'palavras_mais_comuns': palavras_mais_comuns,
        'comprimento_medio_palavras': comprimento_medio_palavras,
        'contagem_pontuacao': contagem_pontuacao
    }

# Módulo de palavras mágicas
def contar_palavras_magicas(texto, palavras_magicas):
    contagem_palavras_magicas = {}
    for palavra in palavras_magicas:
        contagem_palavras_magicas[palavra] = texto.lower().count(palavra.lower())
    return contagem_palavras_magicas

# Módulo de porcentagem de palavras mágicas
def calcular_porcentagem_palavras_magicas(contagem_palavras_magicas, total_palavras):
    porcentagem_palavras_magicas = {}
    for palavra, contagem in contagem_palavras_magicas.items():
        porcentagem_palavras_magicas[palavra] = (contagem / total_palavras) * 100
    return porcentagem_palavras_magicas

# Módulo de palavras por minuto
def calcular_pmm(texto, duracao):
    total_palavras = len(texto.split())
    return total_palavras / (duracao / 60)

# Módulo de palavras desconhecidas
def identificar_palavras_desconhecidas(texto, vocabulario):
    palavras_desconhecidas = []
    for palavra in texto.split():
        if palavra.lower() not in vocabulario:
            palavras_desconhecidas.append(palavra)
    return palavras_desconhecidas

#Main
arquivos_de_audio_uploadados = selecionar_arquivo()
if arquivos_de_audio_uploadados:
    for arquivo_de_audio in arquivos_de_audio_uploadados:
        texto, duracao = transcrever_e_analisar_audio(arquivo_de_audio)
        if texto:
            estatisticas = calcular_estatisticas(texto)
            palavras_magicas = [
                "obrigado", "obrigada", "por favor", "desculpa", "desculpe",
                "por gentileza", "bom dia", "boa tarde", "boa noite", "agradeço",
                "agradece", "gratidão", "sinto muito", "perdão", "me perdoe", "com licença"
            ]
            contagem_palavras_magicas = contar_palavras_magicas(texto, palavras_magicas)
            porcentagem_palavras_magicas = calcular_porcentagem_palavras_magicas(contagem_palavras_magicas, estatisticas["total_palavras"])

            # Imprimir estatísticas
            print("\nEstatísticas da Transcrição:")
            print(f"Total de palavras: {estatisticas['total_palavras']}")
            print("Palavras mais comuns:")
            for palavra, frequencia in estatisticas["palavras_mais_comuns"]:
                print(f"- {palavra}: {frequencia} vezes")
            print(f"Comprimento médio das palavras: {estatisticas['comprimento_medio_palavras']:.2f}")
            print(f"Contagem de pontuação: {estatisticas['contagem_pontuacao']}")
            print(f"Palavras por minuto: {calcular_pmm(texto, duracao):.2f}")
            print("Palavras Mágicas:")
            for palavra, contagem in contagem_palavras_magicas.items():
                print(f"- {palavra}: {contagem} vezes ({porcentagem_palavras_magicas[palavra]:.2f}%)")

            vocabulario = palavras_magicas
            palavras_desconhecidas = identificar_palavras_desconhecidas(texto, vocabulario)
            print(f"Palavras Desconhecidas: {', '.join(palavras_desconhecidas)}")


Estatísticas da Transcrição:
Total de palavras: 617
Palavras mais comuns:
- a: 47 vezes
- que: 17 vezes
- o: 16 vezes
- de: 12 vezes
- sara: 11 vezes
- bom: 10 vezes
- do: 10 vezes
- não: 10 vezes
- eu: 9 vezes
- então: 9 vezes
Comprimento médio das palavras: 4.60
Contagem de pontuação: 184
Palavras por minuto: 124.17
Palavras Mágicas:
- obrigado: 0 vezes (0.00%)
- obrigada: 2 vezes (0.32%)
- por favor: 0 vezes (0.00%)
- desculpa: 0 vezes (0.00%)
- desculpe: 0 vezes (0.00%)
- por gentileza: 0 vezes (0.00%)
- bom dia: 4 vezes (0.65%)
- boa tarde: 0 vezes (0.00%)
- boa noite: 0 vezes (0.00%)
- agradeço: 1 vezes (0.16%)
- agradece: 0 vezes (0.00%)
- gratidão: 0 vezes (0.00%)
- sinto muito: 0 vezes (0.00%)
- perdão: 0 vezes (0.00%)
- me perdoe: 0 vezes (0.00%)
- com licença: 0 vezes (0.00%)
Palavras Desconhecidas: <p>Candinavia,, Natalia,, bom, dia., Bom, dia,, Natalia,, Beatriz, da, Toto, Falano., Gostaria, de, falar, com, a, Sara, Silvane, do, TI., Seria, ela, ainda, a, responsável, pel